**Feature Extraction**

In [0]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import string
import gzip
import pandas as pd
from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split
from sklearn import svm
import numpy as np
import nltk
nltk.download('wordnet_ic')
from nltk.corpus import wordnet as wn

from nltk.corpus import stopwords

from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from nltk.corpus import wordnet_ic
traindata = pd.read_csv('/content/drive/My Drive/train.csv')
testdata = pd.read_csv('/content/drive/My Drive/test.csv')
print(traindata.columns)
print(testdata.columns)
ids = list(traindata['id'])
target_train=list(traindata['target'])
train_text = traindata['text']
test_text = testdata['text']
co_ex_tr=[]
co_qe_tr=[]
co_h_tr=[]
co_word=[]
co_letters=[]
co_url_tr=[]
co_at_tr=[]
co_upper_tr=[]
for i in train_text:
    co_ex_tr.append(i.count('!'))
    co_qe_tr.append(i.count('?'))
    co_h_tr.append(i.count('#'))
    co_letters.append(len(i))
    w = i.split(' ')
    co_word.append(len(w))
    co_url_tr.append(i.count('http:'))
    co_at_tr.append(i.count('@'))
    c=0
    for j in i:
        if(j.isupper()):
            c=c+1
    co_upper_tr.append(c)
co_ex_te=[]
co_qe_te=[]
co_h_te=[]
co_word_te=[]
co_letters_te=[]
co_url_te=[]
co_upper_te=[]
co_at_te=[]
for i in test_text:
    co_ex_te.append(i.count('!'))
    co_qe_te.append(i.count('?'))
    co_h_te.append(i.count('#'))
    co_letters_te.append(len(i))
    co_url_te.append(i.count('http:'))
    co_at_te.append(i.count('@'))
    w = i.split(' ')
    co_word_te.append(len(w))
    c=0
    for j in i:
        if(j.isupper()):
            c=c+1
    co_upper_te.append(c)
        
#traindata['keyword'].fillna('blank')
#testdata['keyword'].fillna('blank')
#traindata['location'].fillna('blank')
#testdata['location'].fillna('blank')
traindata = traindata.drop(columns=['keyword','location','id','target','text'],axis=0)
testdata = testdata.drop(columns=['keyword','location','id','text'],axis=0)
traindata['Count of ex_mark'] = co_ex_tr
traindata['Count_or_question_mark'] = co_qe_tr
traindata['Count_of_#'] = co_h_tr
traindata['Length in characters'] = co_letters
traindata['Length in words'] = co_word
traindata['Count_of_URL'] = co_url_tr
traindata['Count_of_@'] = co_at_tr
testdata['Count of ex_mark'] = co_ex_te
testdata['Count_or_question_mark'] = co_qe_te
testdata['Count_of_#'] = co_h_te
testdata['Length in characters'] = co_letters_te
testdata['Length in words'] = co_word_te
testdata['Count_of_URL'] = co_url_te
testdata['Count_of_@'] = co_at_te
testdata['Count_of_uppercase'] = co_upper_te
traindata['Count_of_uppercase'] = co_upper_tr


brown_ic = wordnet_ic.ic('ic-brown.dat')
ps=PorterStemmer()
table = str.maketrans('', '', string.punctuation)
data = pd.read_csv('/content/drive/My Drive/train.csv')
test_data = pd.read_csv('/content/drive/My Drive/test.csv')
real_word=[]
nonreal_word=[]
for i in range(5000):
    if data['target'][i]==0:
        tokens = nltk.word_tokenize(data['text'][i])
        tokens = [w.lower() for w in tokens]
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokens]
        words = [word for word in stripped if word.isalpha()]
        stop_words = set(stopwords.words('english'))
        words = [w for w in words if not w in stop_words]
        lmt = [wordnet_lemmatizer.lemmatize(word) for word in words]
        lm = [ps.stem(word) for word in lmt]
        for l in lm:
            if l not in nonreal_word:
                if wn.synsets(l,pos='n'):
                    nonreal_word.append(l)
    else:
        tokens = nltk.word_tokenize(data['text'][i])
        tokens = [w.lower() for w in tokens]
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokens]
        words = [word for word in stripped if word.isalpha()]
        stop_words = set(stopwords.words('english'))
        words = [w for w in words if not w in stop_words]
        lmt = [wordnet_lemmatizer.lemmatize(word) for word in words]
        lm = [ps.stem(word) for word in lmt]
        for l in lm:
            if l not in real_word:
                if wn.synsets(l,pos='n'):
                    real_word.append(l)
        
original =[]
computed=[]
real_similarity=[]
fake_similarity=[]
for lk in range(len(data['target'])):
    word=[]
    tokens = nltk.word_tokenize(data['text'][lk])
    tokens = [w.lower() for w in tokens]
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    words = [word for word in stripped if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    lmt = [wordnet_lemmatizer.lemmatize(word) for word in words]
    lm = [ps.stem(word) for word in lmt]
    simi1=[]
    simi2=[]
    for l in lm:
        if l not in word:
            if wn.synsets(l,pos='n'):
                word.append(l)
   
    if len(word)==0:
        real_similarity.append(0)
        fake_similarity.append(0)
        continue
    for i in word:
        try:
            s = wn.synset(i+'.n.01')
        except:
            s = None
            continue
        for k in real_word:
            try:
                s2 = wn.synset(k+'.n.01')
            except:
                s2=None
                continue
            
            sim = s.res_similarity(s2,brown_ic)
            simi1.append(sim)
    #calculate resnik similarity between the synonym or the sense of each word of question 2 with
    #sense of each word of question 1 
    for i in real_word:
        try:
            s = wn.synset(i+'.n.01')
        except:
            s = None
            continue
        for k in word:
            try:
                s2 = wn.synset(k+'.n.01')
            except:
                s2=None
                continue
            sim = s.res_similarity(s2,brown_ic)
            simi2.append(sim)
    #find the average of the similarities of two words which have the maximum similarity
    if(len(simi1) == 0 and len(simi2)==0):
        similarity1=0
    elif(len(simi1) == 0):
        similarity1 = (max(simi2))/2
    elif(len(simi2)==0):
        similarity1 = (max(simi1))/2
    else:
        similarity1 = (max(simi1)+max(simi2))/2
    real_similarity.append(similarity1)
    simi1=[]
    simi2=[]
    for i in word:
        try:
            s = wn.synset(i+'.n.01')
        except:
            s = None
            continue
        for k in nonreal_word:
            try:
                s2 = wn.synset(k+'.n.01')
            except:
                s2=None
                continue
            sim = s.res_similarity(s2,brown_ic)
            simi1.append(sim)
    #calculate resnik similarity between the synonym or the sense of each word of question 2 with
    #sense of each word of question 1 
    for i in nonreal_word:
        try:
            s = wn.synset(i+'.n.01')
        except:
            s = None
            continue
        for k in word:
            try:
                s2 = wn.synset(k+'.n.01')
            except:
                s2=None
                continue
            sim = s.res_similarity(s2,brown_ic)
            simi2.append(sim)
    #find the average of the similarities of two words which have the maximum similarity
    if(len(simi1) == 0 and len(simi2)==0):
        similarity2=0
    elif(len(simi1) == 0):
        similarity2 = (max(simi2))/2
    elif(len(simi2)==0):
        similarity2 = (max(simi1))/2
    else:
        similarity2 = (max(simi1)+max(simi2))/2
    fake_similarity.append(similarity2)
traindata['Resnik_Similarity_real'] = real_similarity
traindata['Resnik_Similarity_fake'] = fake_similarity
real_similarity_te=[]
fake_similarity_te=[]
for lk in range(len(test_data['text'])):
    word=[]
    tokens = nltk.word_tokenize(test_data['text'][lk])
    tokens = [w.lower() for w in tokens]
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    words = [word for word in stripped if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    lmt = [wordnet_lemmatizer.lemmatize(word) for word in words]
    lm = [ps.stem(word) for word in lmt]
    simi1=[]
    simi2=[]
    for l in lm:
        if l not in word:
            if wn.synsets(l,pos='n'):
                word.append(l)
   
    if len(word)==0:
        real_similarity_te.append(0)
        fake_similarity_te.append(0)
        continue
    for i in word:
        try:
            s = wn.synset(i+'.n.01')
        except:
            s = None
            continue
        for k in real_word:
            try:
                s2 = wn.synset(k+'.n.01')
            except:
                s2=None
                continue
            
            sim = s.res_similarity(s2,brown_ic)
            simi1.append(sim)
    #calculate resnik similarity between the synonym or the sense of each word of question 2 with
    #sense of each word of question 1 
    for i in real_word:
        try:
            s = wn.synset(i+'.n.01')
        except:
            s = None
            continue
        for k in word:
            try:
                s2 = wn.synset(k+'.n.01')
            except:
                s2=None
                continue
            sim = s.res_similarity(s2,brown_ic)
            simi2.append(sim)
    #find the average of the similarities of two words which have the maximum similarity
    if(len(simi1) == 0 and len(simi2)==0):
        similarity1=0
    elif(len(simi1) == 0):
        similarity1 = (max(simi2))/2
    elif(len(simi2)==0):
        similarity1 = (max(simi1))/2
    else:
        similarity1 = (max(simi1)+max(simi2))/2
    real_similarity_te.append(similarity1)
    simi1=[]
    simi2=[]
    for i in word:
        try:
            s = wn.synset(i+'.n.01')
        except:
            s = None
            continue
        for k in nonreal_word:
            try:
                s2 = wn.synset(k+'.n.01')
            except:
                s2=None
                continue
            sim = s.res_similarity(s2,brown_ic)
            simi1.append(sim)
    #calculate resnik similarity between the synonym or the sense of each word of question 2 with
    #sense of each word of question 1 
    for i in nonreal_word:
        try:
            s = wn.synset(i+'.n.01')
        except:
            s = None
            continue
        for k in word:
            try:
                s2 = wn.synset(k+'.n.01')
            except:
                s2=None
                continue
            sim = s.res_similarity(s2,brown_ic)
            simi2.append(sim)
    #find the average of the similarities of two words which have the maximum similarity
    if(len(simi1) == 0 and len(simi2)==0):
        similarity2=0
    elif(len(simi1) == 0):
        similarity2 = (max(simi2))/2
    elif(len(simi2)==0):
        similarity2 = (max(simi1))/2
    else:
        similarity2 = (max(simi1)+max(simi2))/2
    fake_similarity_te.append(similarity2)
testdata['Resnik_Similarity_real'] = real_similarity_te
testdata['Resnik_Similarity_fake'] = fake_similarity_te
traindata.to_csv("train_data.csv",index=false)
testdata.to_csv("test_data.csv",index=false)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet_ic to /root/nltk_data...
[nltk_data]   Package wordnet_ic is already up-to-date!
Index(['id', 'keyword', 'location', 'text', 'target'], dtype='object')
Index(['id', 'keyword', 'location', 'text'], dtype='object')


NameError: ignored

In [0]:
traindata.to_csv('/content/drive/My Drive/Train_data.csv',index=False)
testdata.to_csv('/content/drive/My Drive/Test_data.csv',index=False)

In [0]:
new_data_tr = pd.read_csv('/content/drive/My Drive/Train_data_New.csv')
new_data_te = pd.read_csv('/content/drive/My Drive/Test_data_New.csv')
traindata = pd.read_csv()
for i in traindata.columns:
  if i in new_data_tr.columns:
    continue
  else:
    new_data_tr[i] = traindata[i]
for i in testdata.columns:
  if i in new_data_te.columns:
    continue
  else:
    new_data_te[i] = testdata[i]
print(new_data_tr)
print(new_data_te)
new_data_te.to_csv('/content/drive/My Drive/Train_Data.csv',index=False)
new_data_tr.to_csv('/content/drive/My Drive/Test_Data.csv',index=False)

         id keyword  ... Resnik_Similarity_real Resnik_Similarity_fake
0         1     NaN  ...           1.446560e+01           1.446560e+01
1         4     NaN  ...           1.091025e+01           1.091025e+01
2         5     NaN  ...          1.000000e+300           9.840627e+00
3         6     NaN  ...           1.021710e+01           1.021710e+01
4         7     NaN  ...           1.137456e+01           1.137456e+01
...     ...     ...  ...                    ...                    ...
7608  10869     NaN  ...          1.000000e+300          1.000000e+300
7609  10870     NaN  ...           1.068141e+01           1.040516e+01
7610  10871     NaN  ...          1.000000e+300          1.000000e+300
7611  10872     NaN  ...           1.047662e+01           1.047662e+01
7612  10873     NaN  ...          1.000000e+300          1.000000e+300

[7613 rows x 19 columns]
         id keyword  ... Resnik_Similarity_real Resnik_Similarity_fake
0         0     NaN  ...           1.113340e+01    